In [11]:
import requests

def describe_metal(metal_name: str) -> str:
    """
    Tries to fetch a description of the metal from Wikipedia.
    Falls back from 'metal_name (metal)' to just 'metal_name' if necessary.
    """
    def fetch_summary(title: str):
        url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title.replace(' ', '_')}"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        return data.get("extract", "No summary found.")
    
    try:
        # Primero intenta con "(metal)"
        return fetch_summary(f"{metal_name} (metal)")
    except requests.HTTPError:
        try:
            # Si falla, prueba con el nombre base
            return fetch_summary(metal_name)
        except Exception as e2:
            return f"Error fetching metal description: {e2}"
    except Exception as e:
        return f"Error fetching metal description: {e}"

In [8]:
import giskard
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from giskard import Model, Dataset

# Dataset ficticio con columna sensible 'gender'
data = pd.DataFrame({
    'age': [25, 45, 35, 23, 52, 46, 33, 29],
    'income': [40000, 80000, 50000, 30000, 90000, 85000, 45000, 42000],
    'gender': ['F', 'M', 'F', 'F', 'M', 'M', 'F', 'M'],
    'default': [0, 1, 0, 0, 1, 1, 0, 0]  # variable objetivo
})

X = data[['age', 'income']]
y = data['default']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
model = RandomForestClassifier().fit(X_train, y_train)

# Convertimos a predictor de Giskard
giskard_model = giskard.Model(
    model=model,
    model_type="classification",
    name="rf_default_predictor",
    feature_names=["age", "income"],
    classification_labels=[0, 1]
)

# Creamos el dataset Giskard (agregamos la variable sensible)
giskard_dataset = giskard.Dataset(
    df=X_test.assign(default=y_test.values, gender=data.loc[X_test.index, 'gender']),
    target="default",
    cat_columns=["gender"]
)

# Define una test suite (usa tests predefinidos como ejemplo)
test_suite = TestSuite(name="My Test Suite")
test_suite.add_test(testing.test_fairness(gender="gender"))

# Ejecuta la suite con tu modelo y dataset
results = test_suite.run(model=giskard_model, dataset=giskard_dataset)

# Itera sobre los resultados
for result in results.results:
    print(f"Test: {result.meta.name}")
    print(f"   Passed: {result.passed}")
    print(f"   Message: {result.message}")
    print()



2025-06-11 13:24:56,023 pid:7216 MainThread giskard.models.automodel INFO     Your 'model' is successfully wrapped by Giskard's 'SKLearnModel' wrapper class.
2025-06-11 13:24:56,035 pid:7216 MainThread giskard.datasets.base INFO     Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.


NameError: name 'TestSuite' is not defined

In [14]:
import giskard
import pandas as pd
from IPython.display import Image, display

# Simulamos preguntas, contexto y respuestas del agente
df = pd.DataFrame([
    {
        "question": "What is platinum used for?",
        "context": describe_metal("platinum"),
        "agent_output": "Platinum is mainly used in catalytic converters and electrical contacts."
    },
    {
        "question": "Describe copper applications",
        "context": describe_metal("copper"),
        "agent_output": "Copper is used in wiring and plumbing systems."
    }
])


ImportError: cannot import name using_string_dtype

In [13]:
def rag_agent_model(df):
    return df["agent_output"]

wrapped_model = giskard.Model(
    model=rag_agent_model,
    model_type="text_generation",
    name="rag_agent"
)
wrapped_dataset = giskard.Dataset(df, name="MetalQA", target="agent_output")
@giskard.test()
def response_mentions_application(df):
    for output in df["agent_output"]:
        assert any(word in output.lower() for word in ["used", "application", "purpose"]), \
            f"Output lacks usage mention: {output}"
result = response_mentions_application(wrapped_dataset)
result.display()


NameError: name 'giskard' is not defined